In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp
import os
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

lorenz func data gen

outputs csvs with format time, Var where time is an integer, and Var is the value of given Var at t

In [ ]:


def lorenz(t, state, sigma=10, beta=8/3, rho=28):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Initial conditions
state0 = [1.0, 1.0, 1.0]
time_span = (0, 10000)


# Generate integer time points
#time_eval = np.linspace(time_span[0], time_span[1], 10000) #Original Line
time_eval = np.arange(time_span[0], time_span[1] + 1, dtype=int)  # New Line

output_dir = 'lorenz_output'
if not os.path.exists(output_dir):
       os.makedirs(output_dir)


for i in range(-2000,2000):

    #Scrambles x y and z starting conditions to generate a new system
    state0[i % 3] == i / 100000

    # Solve the system
    sol = solve_ivp(lorenz, time_span, state0, t_eval=time_eval)

    #Save the time series data of the x y and z axes as individual csvs
    dfx = pd.DataFrame({'Time': sol.t, 'X': sol.y[0]})
    dfx.to_csv("lorenz_output_x" + str(i) + ".csv", index=False)
    print("Data saved to lorenz_output_x" + str(i) + ".csv")

    dfy = pd.DataFrame({'Time': sol.t, 'Y': sol.y[1]})
    dfy.to_csv("lorenz_output_y " + str(i) + ".csv", index=False)
    print("Data saved to lorenz_output_y" + str(i) + ".csv")

    dfz = pd.DataFrame({'Time': sol.t, 'Z': sol.y[2]})
    dfz.to_csv("lorenz_output_z" + str(i) + ".csv", index=False)
    print("Data saved to lorenz_output_z" + str(i) + ".csv")

In [ ]:
import pandas as pd
import glob
import os

def combine_csv_files(directory, output_prefix):
    """Combines CSV files in a directory based on their prefixes (x, y, z)."""

    for prefix in ['x', 'y', 'z']:
        all_files = glob.glob(os.path.join(directory, f'lorenz_output_{prefix}*.csv'))
        if not all_files:
            print(f"No files found with prefix '{prefix}' in the specified directory.")
            continue

        combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
        output_filename = os.path.join(directory, f'{output_prefix}_{prefix}.csv')
        combined_df.to_csv(output_filename, index=False)
        print(f"Combined '{prefix}' files into '{output_filename}'")

# Example usage:
combine_csv_files('/content', 'combined_lorenz')


In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length  # Changed here

    def __getitem__(self, index):
        x = self.data[index:index + self.sequence_length]  # Input: Past sequence_length values
        y = self.data[index + self.sequence_length]  # Target: The next value after the sequence
        return x, y


# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):  # Changed input parameters
        super(MLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # Input layer with input_size features
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),  # Hidden layer with hidden_size features
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)   # Output layer with output_size features
        )

    def forward(self, x):
        return self.network(x)


# Training function (remains the same)
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        for data, targets in dataloader:
            # Forward pass
            outputs = model(data).squeeze(1)  # Squeeze the output tensor along dimension 1
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Loading Data

In [ ]:
dfx = pd.read_csv("/content/combined_lorenz_x.csv")  # Load data for 'x'
data = torch.tensor(dfx['X'].values, dtype=torch.float32)
targets = torch.randn(100, 1)  # 100 targets
sequence_length = 5
# Hyperparameters
hidden_size = 50
output_size = 1
num_epochs = 10
batch_size = 10
learning_rate = 0.1
sequence_length = 20  # Length of the input sequence for time-series prediction
# Dataset and DataLoader
dataset = TimeSeriesDataset(data,sequence_length)  # Only pass data here
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, criterion, and optimizer
model = MLP(sequence_length, hidden_size, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for batch_idx, (data, targets) in enumerate(dataloader):
    print(f"Batch {batch_idx}:")
    print("Data:", data)
    print("Targets:", targets)
    break  # Stop after the first batch to avoid printing too much

# training

In [ ]:
train_model(model, dataloader, criterion, optimizer, num_epochs)

Epoch [1/10], Loss: 73.0187
Epoch [2/10], Loss: 78.2009
